In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
from secrets import IEX_CLOUD_API_TOKEN

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 99684658,
 'calculationPrice': 'tops',
 'change': 2.67,
 'changePercent': 0.01798,
 'close': 0,
 'closeSource': 'ciliafof',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 159.6,
 'iexAskSize': 203,
 'iexBidPrice': 160.2,
 'iexBidSize': 100,
 'iexClose': 160.35,
 'iexCloseTime': 1706344275714,
 'iexLastUpdated': 1672809974309,
 'iexMarketPercent': 0.025419036688515678,
 'iexOpen': 155.94,
 'iexOpenTime': 1679469617221,
 'iexRealtimePrice': 157.05,
 'iexRealtimeSize': 104,
 'iexVolume': 2528165,
 'lastTradeTime': 1680280117482,
 'latestPrice': 159.86,
 'latestSource': 'IEX real time price',
 'latestTime': '3:39:11 PM',
 'latestUpdate': 1702158085230,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'm

In [4]:
# PRICE TO EARNINGS RATIO
pe_ratio = data['peRatio']
pe_ratio

25.41

In [5]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ],
                                                  index=my_columns),
                                        ignore_index=True)
final_df

/tmp/ipykernel_579/3800848382.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_579/3800848382.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_579/3800848382.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_579/3800848382.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/ipykernel_579/3800848382.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/tmp/

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,119.02,31.07,N/A
1,AAL,17.19,-4.59,N/A
2,AAP,203.96,21.45,N/A
3,AAPL,161.27,26.33,N/A
4,ABBV,155.08,22.12,N/A
...,...,...,...,...
496,YUM,113,22.13,N/A
497,ZBH,117.2,116.64,N/A
498,ZBRA,333.5,21.37,N/A
499,ZION,53.82,5.72,N/A


In [7]:
# GETTING TOP 30 VALUE STOCKS AND REMOVING GLAMOUR STOCKS
final_df.sort_values('Price-to-Earnings Ratio', inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:30]
final_df.sort_values(by='Price-to-Earnings Ratio', ascending=False, inplace=True)
final_df.reset_index(inplace=True)
final_df.drop('index', axis=1, inplace=True)

In [8]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WU,16.79,7.63,N/A
1,CE,140.88,7.55,N/A
2,DOW,67.352,7.47,N/A
3,INTC,44.95,7.44,N/A
4,WHR,192.09,7.14,N/A
5,HPQ,37.968,6.95,N/A
6,GM,39.3,6.75,N/A
7,BEN,25.55,6.65,N/A
8,IVZ,17.8,6.36,N/A
9,DISH,22.46,6.34,N/A


In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
portfolio_input()

In [10]:
print(portfolio_size)

100000


In [11]:
# DOMINATED PE_RATIO STRATEGY
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df['Price'][i])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WU,16.79,7.63,198
1,CE,140.88,7.55,23
2,DOW,67.352,7.47,49
3,INTC,44.95,7.44,74
4,WHR,192.09,7.14,17
5,HPQ,37.968,6.95,87
6,GM,39.3,6.75,84
7,BEN,25.55,6.65,130
8,IVZ,17.8,6.36,187
9,DISH,22.46,6.34,148


In [12]:
# SUPERIOR COMPOSITE VALUE STRATEGY
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

/tmp/ipykernel_579/2824602555.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/tmp/ipykernel_579/2824602555.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/tmp/ipykernel_579/2824602555.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/tmp/ipykernel_579/2824602555.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/tmp/ipykernel_579/2824602555.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [14]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,118.96,N/A,30.09,N/A,6.72,N/A,5.4,N/A,20.315954,N/A,10.07592,N/A,N/A
1,AAL,16.81,N/A,-4.6,N/A,-1.2,N/A,0.309,N/A,-23.431001,N/A,1.071837,N/A,N/A
2,AAP,211.8,N/A,21.4,N/A,3.98,N/A,1.18,N/A,11.69952,N/A,2.529085,N/A,N/A
3,AAPL,158.91,N/A,25.77,N/A,36.94,N/A,6.39,N/A,19.632141,N/A,15.196204,N/A,N/A
4,ABBV,153.56,N/A,22.28,N/A,16.91,N/A,4.9,N/A,11.363754,N/A,8.463273,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.46,N/A,22.2,N/A,-3.82,N/A,4.86,N/A,18.703606,N/A,8.727573,N/A,N/A
497,ZBH,118.06,N/A,115.57,N/A,2.03,N/A,3.19,N/A,18.924068,N/A,5.779034,N/A,N/A
498,ZBRA,333.65,N/A,22.12,N/A,5.99,N/A,3.1,N/A,17.410861,N/A,6.985025,N/A,N/A
499,ZION,55.1,N/A,5.7,N/A,1.5,N/A,2.48,N/A,4.575792,N/A,2.345519,N/A,N/A


In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,182.6,N/A,62.37,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,287.56,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,315.319,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11,N/A,10,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.46,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.43,N/A,14.34,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,59.33,N/A,32.97,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,31.13,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,112.56,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
203,GOOG,2367.09,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

for metric in metrics.values():
    print(rv_dataframe[metric])

rv_dataframe

0      0.714571
1      0.033932
2      0.506986
3      0.622754
4       0.53493
         ...   
496    0.530938
497    0.984032
498    0.526946
499    0.070858
500    0.822355
Name: PE Percentile, Length: 501, dtype: object
0      0.787425
1      0.061876
2      0.600798
3       0.97006
4      0.932136
         ...   
496    0.057884
497    0.328343
498    0.746507
499    0.218563
500    0.928144
Name: PB Percentile, Length: 501, dtype: object
0      0.795409
1      0.013972
2      0.173653
3      0.842315
4      0.750499
         ...   
496    0.744511
497    0.536926
498    0.526946
499    0.423154
500    0.926148
Name: PS Percentile, Length: 501, dtype: object
0      0.788423
1      0.007984
2      0.367265
3      0.758483
4      0.345309
         ...   
496    0.718563
497    0.724551
498    0.666667
499     0.06986
500    0.896208
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.750499
1      0.031936
2      0.141717
3      0.916168
4      0.652695
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,118.96,N/A,30.09,0.714571,6.72,0.787425,5.400,0.795409,20.315954,0.788423,10.075920,0.750499,N/A
1,AAL,16.81,N/A,-4.60,0.033932,-1.20,0.061876,0.309,0.013972,-23.431001,0.007984,1.071837,0.031936,N/A
2,AAP,211.8,N/A,21.40,0.506986,3.98,0.600798,1.180,0.173653,11.699520,0.367265,2.529085,0.141717,N/A
3,AAPL,158.91,N/A,25.77,0.622754,36.94,0.97006,6.390,0.842315,19.632141,0.758483,15.196204,0.916168,N/A
4,ABBV,153.56,N/A,22.28,0.53493,16.91,0.932136,4.900,0.750499,11.363754,0.345309,8.463273,0.652695,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.46,N/A,22.20,0.530938,-3.82,0.057884,4.860,0.744511,18.703606,0.718563,8.727573,0.664671,N/A
497,ZBH,118.06,N/A,115.57,0.984032,2.03,0.328343,3.190,0.536926,18.924068,0.724551,5.779034,0.41517,N/A
498,ZBRA,333.65,N/A,22.12,0.526946,5.99,0.746507,3.100,0.526946,17.410861,0.666667,6.985025,0.502994,N/A
499,ZION,55.1,N/A,5.70,0.070858,1.50,0.218563,2.480,0.423154,4.575792,0.06986,2.345519,0.131737,N/A


In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,118.96,N/A,30.09,0.714571,6.72,0.787425,5.400,0.795409,20.315954,0.788423,10.075920,0.750499,0.767265
1,AAL,16.81,N/A,-4.60,0.033932,-1.20,0.061876,0.309,0.013972,-23.431001,0.007984,1.071837,0.031936,0.02994
2,AAP,211.8,N/A,21.40,0.506986,3.98,0.600798,1.180,0.173653,11.699520,0.367265,2.529085,0.141717,0.358084
3,AAPL,158.91,N/A,25.77,0.622754,36.94,0.97006,6.390,0.842315,19.632141,0.758483,15.196204,0.916168,0.821956
4,ABBV,153.56,N/A,22.28,0.53493,16.91,0.932136,4.900,0.750499,11.363754,0.345309,8.463273,0.652695,0.643114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.46,N/A,22.20,0.530938,-3.82,0.057884,4.860,0.744511,18.703606,0.718563,8.727573,0.664671,0.543313
497,ZBH,118.06,N/A,115.57,0.984032,2.03,0.328343,3.190,0.536926,18.924068,0.724551,5.779034,0.41517,0.597804
498,ZBRA,333.65,N/A,22.12,0.526946,5.99,0.746507,3.100,0.526946,17.410861,0.666667,6.985025,0.502994,0.594012
499,ZION,55.1,N/A,5.70,0.070858,1.50,0.218563,2.480,0.423154,4.575792,0.06986,2.345519,0.131737,0.182834


In [20]:
rv_dataframe.sort_values(by = 'RV Score', inplace=True, ascending=False)
rv_dataframe = rv_dataframe[:30]
rv_dataframe.reset_index(drop = True, inplace = True)

In [21]:
portfolio_input()

In [22]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,449.38,66,403.30,1.0,22.12,0.954092,13.70,0.97006,116.336488,0.998004,17.045542,0.942116,0.972854
1,DXCM,335.59,89,164.18,0.992016,15.10,0.924152,12.73,0.962076,83.742851,0.994012,18.715088,0.958084,0.966068
2,PAYC,294.35,101,77.80,0.96008,16.87,0.93014,15.46,0.982036,46.442858,0.98004,16.732739,0.936128,0.957685
3,NVDA,182.25,164,46.13,0.88024,18.78,0.94012,18.71,0.996008,43.614864,0.976048,28.036078,0.992016,0.956886
4,FTNT,263.94,113,71.50,0.952096,188.78,0.996008,11.41,0.948104,48.249698,0.982036,13.603919,0.884232,0.952495
5,CDNS,148.36,202,54.29,0.926148,14.38,0.92016,12.38,0.956088,40.609194,0.972056,12.767081,0.858283,0.926547
6,ADSK,191.38,156,85.69,0.968064,50.12,0.982036,9.69,0.92016,58.204804,0.988024,10.399283,0.768463,0.925349
7,MA,343.13,87,35.50,0.800399,46.13,0.976048,16.51,0.992016,27.718840,0.9002,16.879539,0.94012,0.921756
8,TYL,364.27,82,93.61,0.974052,6.31,0.756487,8.46,0.894212,62.732238,0.992016,21.217614,0.97006,0.917365
9,IDXX,339.84,88,41.54,0.858283,47.00,0.978044,9.01,0.908184,28.691183,0.918164,15.766940,0.92016,0.916567


In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()